<a href="https://colab.research.google.com/github/VytautasBruzila/Baigiamasis-darbas-JackDaniels/blob/main/Final_Version_of_Jack_Daniels_fake_bottle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import os
import cv2
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import img_to_array

# Define the paths to the original datasets
fake_dir = '/content/drive/MyDrive/datasets/fake'
original_dir = '/content/drive/MyDrive/datasets/original'

# Define the paths to the preprocessed datasets
preprocessed_fake_dir = '/content/drive/MyDrive/datasets/preprocessed/fake'
preprocessed_original_dir = '/content/drive/MyDrive/datasets/preprocessed/original'

# Create the preprocessed directories if they don't exist
os.makedirs(preprocessed_fake_dir, exist_ok=True)
os.makedirs(preprocessed_original_dir, exist_ok=True)

# Define the target size for the images
target_size = (512, 512)

# Function to preprocess an image
def preprocess_image(image_path, output_dir, label):
    # Load the image
    image = cv2.imread(image_path)

    # Resize the image
    resized_image = cv2.resize(image, target_size)

    # Convert to grayscale
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Normalize the image
    normalized_image = preprocess_input(grayscale_image)

    # Convert to NumPy array
    numpy_image = img_to_array(normalized_image)

    # Save the preprocessed image
    output_path = os.path.join(output_dir, f"{label}_{os.path.basename(image_path)}")
    cv2.imwrite(output_path, numpy_image)

# Preprocess the fake images
for filename in os.listdir(fake_dir):
    image_path = os.path.join(fake_dir, filename)
    preprocess_image(image_path, preprocessed_fake_dir, 'fake')

# Preprocess the original images
for filename in os.listdir(original_dir):
    image_path = os.path.join(original_dir, filename)
    preprocess_image(image_path, preprocessed_original_dir, 'original')

In [22]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Initialize the CNN
model = Sequential()

# Add the convolutional layer
model.add(Conv2D(32, (3, 3), input_shape=(512, 512, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())

# Full connection
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the CNN
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Part 2 - Fitting the CNN to the images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/datasets/preprocessed',
    target_size=(512, 512),
    batch_size=32,
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/datasets/preprocessed',
    target_size=(512, 512),
    batch_size=32,
    class_mode='binary'
)

model.fit_generator(
    training_set,
    steps_per_epoch=len(training_set) // 8,
    epochs=25,
    validation_data=test_set,
    validation_steps=len(test_set) // 8
)

# Part 3 - Making new predictions
def predict_image(file):
    test_image = image.load_img(file, target_size=(512, 512))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)
    if result[0][0] >= 0.5:
        prediction = 'fake'
    else:
        prediction = 'original'
    return prediction

Found 417 images belonging to 2 classes.
Found 417 images belonging to 2 classes.


<ipython-input-22-54d1cd826019>:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/25
1/1 [==============================] - 34s 34s/step - loss: 0.7289 - accuracy: 0.3125 - val_loss: 25.6385 - val_accuracy: 0.6250
Epoch 2/25
1/1 [==============================] - 11s 11s/step - loss: 8.3109e-38 - accuracy: 1.0000 - val_loss: 44.2613 - val_accuracy: 0.6250
Epoch 3/25
1/1 [==============================] - 31s 31s/step - loss: 25.4691 - accuracy: 0.7812 - val_loss: 19.0746 - val_accuracy: 0.5938
Epoch 4/25
1/1 [==============================] - 39s 39s/step - loss: 22.5500 - accuracy: 0.5312 - val_loss: 3.9781 - val_accuracy: 0.2812
Epoch 5/25
1/1 [==============================] - 31s 31s/step - loss: 3.8092 - accuracy: 0.3125 - val_loss: 0.0952 - val_accuracy: 0.9688
Epoch 6/25
1/1 [==============================] - 31s 31s/step - loss: 0.0495 - accuracy: 0.9688 - val_loss: 0.5617 - val_accuracy: 0.6875
Epoch 7/25
1/1 [==============================] - 32s 32s/step - loss: 0.5570 - accuracy: 0.6562 - val_loss: 0.1225 - val_accuracy: 1.0000
Epoch 8/25
1/1 [==

In [52]:
def upload_file_for_prediction():
    file_path = input("Enter the file path for prediction: ")
    if file_path:
        # Import necessary modules within the function
        from keras.preprocessing import image
        import numpy as np

        # Predict image class
        test_image = image.load_img(file_path, target_size=(512, 512))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis=0)
        result = model.predict(test_image)
        if result[0][0] >= 0.1:
            prediction = 'Original'
        else:
            prediction = 'Fake'

        print('File "{}" is predicted as: {}'.format(file_path, prediction))
    else:
        print("No file path entered.")

# Call the function to upload file for prediction
upload_file_for_prediction()

Enter the file path for prediction: /content/drive/MyDrive/datasets/fake/How to identify fake Jack Daniels_000060.jpg
1/1 [==============================] - 0s 186ms/step
File "/content/drive/MyDrive/datasets/fake/How to identify fake Jack Daniels_000060.jpg" is predicted as: Fake


This tool make the checking procedure much more simplier.